# Active Sensing Feature Extraction

## Introduction

This example demonstrates active sensing feature extraction using guided wave analysis. The approach uses ultrasonic waves transmitted through the structure and analyzes scattered signals to detect damage.

Data from the **active sensing dataset** are used to demonstrate:
- Matched filtering for signal detection
- Geometric analysis of propagation paths
- Time-of-flight calculations
- Spatial damage mapping

**Key Concepts:**
- **Guided Wave Propagation**: Ultrasonic waves travel through structure
- **Matched Filtering**: Coherent and incoherent signal detection
- **Line-of-Sight Analysis**: Geometric constraints on wave paths
- **Spatial Imaging**: Mapping damage indicators to 2D space

**References:**

Su, Z., & Ye, L. (2009). Identification of damage using Lamb waves: from fundamentals to applications. Springer Science & Business Media.

**SHMTools functions used:**
- `coherent_matched_filter_shm`
- `incoherent_matched_filter_shm`
- `estimate_group_velocity_shm`
- `propagation_dist_2_points_shm`
- `distance_2_index_shm`
- `build_contained_grid_shm`
- `import_ActiveSense1_shm`

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import sys

# Add shmtools to path - handle different execution contexts
current_dir = Path.cwd()
notebook_dir = Path(__file__).parent if '__file__' in globals() else current_dir

# Try different relative paths to find shmtools
possible_paths = [
    notebook_dir.parent.parent.parent,  # From examples/notebooks/advanced/
    current_dir.parent.parent,          # From examples/notebooks/
    current_dir,                        # From project root
    Path('/Users/eric/repo/shm/shmtools-python')  # Absolute fallback
]

shmtools_found = False
for path in possible_paths:
    if (path / 'shmtools').exists():
        if str(path) not in sys.path:
            sys.path.insert(0, str(path))
        shmtools_found = True
        print(f"Found shmtools at: {path}")
        break

if not shmtools_found:
    print("Warning: Could not find shmtools module")

from shmtools.utils.data_io import import_ActiveSense1_shm
from shmtools.active_sensing import (
    coherent_matched_filter_shm,
    incoherent_matched_filter_shm,
    estimate_group_velocity_shm,
    propagation_dist_2_points_shm,
    distance_2_index_shm,
    build_contained_grid_shm,
    sensor_pair_line_of_sight_shm,
    fill_2d_map_shm
)

# Set up plotting
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# Set random seed for reproducibility
np.random.seed(42)

# Load active sensing data
print("Loading active sensing dataset...")
(
    waveform_base,
    waveform_test, 
    sensor_layout,
    pair_list,
    border_comb,
    sample_rate,
    point_list,
    actuation_freq
) = import_ActiveSense1_shm()

print(f"Data loaded successfully!")
print(f"Baseline waveform shape: {waveform_base.shape}")
print(f"Test waveform shape: {waveform_test.shape}")
print(f"Number of sensors: {sensor_layout.shape[0]}")
print(f"Number of sensor pairs: {pair_list.shape[0]}")
print(f"Sampling rate: {sample_rate:.0f} Hz")
print(f"Actuation frequency: {actuation_freq}")

# Display some key parameters
n_time_points = waveform_base.shape[0]
n_sensors = waveform_base.shape[1] 
time_duration = n_time_points / sample_rate

print(f"\nMeasurement Details:")
print(f"Time duration: {time_duration*1000:.2f} ms")
print(f"Time resolution: {1/sample_rate*1e6:.2f} μs")
print(f"Frequency resolution: {sample_rate/n_time_points:.1f} Hz")

In [ ]:
# Load active sensing dataprint("Loading active sensing dataset...")(    waveform_base,    waveform_test,     sensor_layout,    pair_list,    border_comb,    sample_rate,    point_list,    actuation_freq) = import_ActiveSense1_shm()print(f"Data loaded successfully\!")print(f"Baseline waveform shape: {waveform_base.shape}")print(f"Test waveform shape: {waveform_test.shape}")print(f"Number of sensors: {sensor_layout.shape[0]}")print(f"Number of sensor pairs: {pair_list.shape[0]}")print(f"Sampling rate: {sample_rate:.0f} Hz")print(f"Actuation frequency: {actuation_freq}")# Display some key parametersn_time_points = waveform_base.shape[0]n_sensors = waveform_base.shape[1] time_duration = n_time_points / sample_rateprint(f"Measurement Details:")print(f"Time duration: {time_duration*1000:.2f} ms")print(f"Time resolution: {1/sample_rate*1e6:.2f} μs")print(f"Frequency resolution: {sample_rate/n_time_points:.1f} Hz")

# Estimate group velocity from baseline measurements
print("Estimating guided wave velocity...")

# Use a subset of pairs for velocity estimation (computational efficiency)
n_pairs_for_velocity = min(10, pair_list.shape[0])
velocity_pairs = pair_list[:n_pairs_for_velocity]

# Estimate actuation width from frequency (handle array case)
if hasattr(actuation_freq, '__len__') and len(actuation_freq) > 0:
    actuation_width = 1.0 / actuation_freq[0]  # Use first element if array
else:
    actuation_width = 1.0 / actuation_freq if actuation_freq > 0 else 1e-5  # Simple estimate

# For velocity estimation, we need to use only the relevant sensors
# The waveform data has many channels, but we only have a few actual sensor locations
n_actual_sensors = sensor_layout.shape[0]
waveform_subset = waveform_base[:, :n_actual_sensors]

# Estimate velocity
estimated_velocity, velocity_list = estimate_group_velocity_shm(
    waveform_subset[:, :, np.newaxis],  # Add instance dimension
    velocity_pairs,
    sensor_layout,
    sample_rate,
    actuation_width
)

print(f"\nVelocity Estimation Results:")
print(f"Estimated group velocity: {estimated_velocity:.0f} m/s")
print(f"Number of valid estimates: {len(velocity_list)}")
if len(velocity_list) > 0:
    print(f"Velocity range: {np.min(velocity_list):.0f} - {np.max(velocity_list):.0f} m/s")
    print(f"Velocity std: ±{np.std(velocity_list):.0f} m/s")

# Use reasonable default if estimation fails
if estimated_velocity <= 0 or estimated_velocity > 10000:
    estimated_velocity = 3000  # Typical value for aluminum plate
    print(f"Using default velocity: {estimated_velocity} m/s")

# Visualize velocity estimates
if len(velocity_list) > 1:
    plt.figure(figsize=(10, 4))
    
    plt.subplot(1, 2, 1)
    plt.hist(velocity_list, bins=10, alpha=0.7, color='blue')
    plt.axvline(estimated_velocity, color='red', linestyle='--', 
               label=f'Estimated: {estimated_velocity:.0f} m/s')
    plt.xlabel('Velocity (m/s)')
    plt.ylabel('Count')
    plt.title('Velocity Distribution')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    plt.plot(velocity_list, 'bo-', alpha=0.7)
    plt.axhline(estimated_velocity, color='red', linestyle='--', 
               label=f'Mean: {estimated_velocity:.0f} m/s')
    plt.xlabel('Sensor Pair Index')
    plt.ylabel('Velocity (m/s)')
    plt.title('Velocity by Sensor Pair')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Visualize sensor layout and structure geometry
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot 1: Sensor layout with pairs
ax1.scatter(sensor_layout[:, 0], sensor_layout[:, 1], 
           c='red', s=100, marker='o', label='Sensors', zorder=3)

# Label sensors
for i, (x, y) in enumerate(sensor_layout):
    ax1.annotate(f'S{i+1}', (x, y), xytext=(5, 5), 
                textcoords='offset points', fontsize=8)

# Plot sensor pairs
for i, pair in enumerate(pair_list[:5]):  # Show first 5 pairs for clarity
    sensor1_idx = pair[0] - 1  # Convert to 0-based indexing
    sensor2_idx = pair[1] - 1
    
    x_coords = [sensor_layout[sensor1_idx, 0], sensor_layout[sensor2_idx, 0]]
    y_coords = [sensor_layout[sensor1_idx, 1], sensor_layout[sensor2_idx, 1]]
    
    ax1.plot(x_coords, y_coords, 'b-', alpha=0.5, linewidth=1)

# Plot structure boundary
if border_comb:
    for border_segment in border_comb:
        if len(border_segment) > 1:
            ax1.plot(border_segment[:, 0], border_segment[:, 1], 'k-', linewidth=2)

ax1.set_xlabel('X Position (m)')
ax1.set_ylabel('Y Position (m)')
ax1.set_title('Sensor Layout and Pairs')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_aspect('equal')

# Plot 2: Sample waveforms
time_axis = np.arange(n_time_points) / sample_rate * 1000  # Convert to ms

# Plot baseline and test waveforms for first sensor
ax2.plot(time_axis, waveform_base[:, 0], 'b-', alpha=0.7, 
         label='Baseline (Sensor 1)', linewidth=1)
ax2.plot(time_axis, waveform_test[:, 0], 'r-', alpha=0.7, 
         label='Test (Sensor 1)', linewidth=1)

ax2.set_xlabel('Time (ms)')
ax2.set_ylabel('Amplitude')
ax2.set_title('Example Waveforms')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nGeometry Statistics:")
print(f"Structure dimensions: {np.ptp(sensor_layout[:, 0]):.3f} × {np.ptp(sensor_layout[:, 1]):.3f} m")
print(f"Sensor pair distances: {np.min([np.linalg.norm(sensor_layout[p[0]-1] - sensor_layout[p[1]-1]) for p in pair_list]):.3f} - {np.max([np.linalg.norm(sensor_layout[p[0]-1] - sensor_layout[p[1]-1]) for p in pair_list]):.3f} m")

## Estimate Group Velocity

Analyze the baseline waveforms to estimate the guided wave propagation velocity.

In [ ]:
# Apply matched filtering to detect guided waves
print("Applying matched filtering...")

# Use only the actual sensor channels (not all waveform channels)
n_actual_sensors = sensor_layout.shape[0]
waveform_base_subset = waveform_base[:, :n_actual_sensors]
waveform_test_subset = waveform_test[:, :n_actual_sensors]

# Calculate difference signal (test - baseline)
waveform_diff = waveform_test_subset - waveform_base_subset

# Use baseline signal as template for matched filtering
# Extract template from actuated sensor (typically first sensor)
template_length = min(100, n_time_points // 4)  # Use first part of signal
template = waveform_base_subset[:template_length, 0]  # Use first sensor as template

print(f"Template length: {template_length} samples ({template_length/sample_rate*1000:.2f} ms)")

# Apply coherent matched filter to difference signal
coherent_result = coherent_matched_filter_shm(waveform_diff, template[:, np.newaxis])

# Apply incoherent matched filter to difference signal  
incoherent_result = incoherent_matched_filter_shm(waveform_diff, template[:, np.newaxis])

print(f"Coherent filter output shape: {coherent_result.shape}")
print(f"Incoherent filter output shape: {incoherent_result.shape}")

# Visualize matched filter results
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Template signal
template_time = np.arange(template_length) / sample_rate * 1000
axes[0, 0].plot(template_time, template, 'b-', linewidth=2)
axes[0, 0].set_xlabel('Time (ms)')
axes[0, 0].set_ylabel('Amplitude')
axes[0, 0].set_title('Template Signal (Baseline)')
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Difference signal (sensor 1)
time_axis = np.arange(n_time_points) / sample_rate * 1000  # Convert to ms
axes[0, 1].plot(time_axis, waveform_diff[:, 0], 'r-', alpha=0.7)
axes[0, 1].set_xlabel('Time (ms)')
axes[0, 1].set_ylabel('Amplitude')
axes[0, 1].set_title('Difference Signal (Test - Baseline)')
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Coherent matched filter result
axes[1, 0].plot(time_axis, coherent_result[:, 0], 'g-', alpha=0.7)
axes[1, 0].set_xlabel('Time (ms)')
axes[1, 0].set_ylabel('Amplitude')
axes[1, 0].set_title('Coherent Matched Filter Output')
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Incoherent matched filter result
axes[1, 1].plot(time_axis, incoherent_result[:, 0], 'm-', alpha=0.7)
axes[1, 1].set_xlabel('Time (ms)')
axes[1, 1].set_ylabel('Magnitude')
axes[1, 1].set_title('Incoherent Matched Filter Output')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate signal energy metrics
baseline_energy = np.mean(np.sum(waveform_base_subset**2, axis=0))
test_energy = np.mean(np.sum(waveform_test_subset**2, axis=0))
diff_energy = np.mean(np.sum(waveform_diff**2, axis=0))

print(f"\nSignal Energy Analysis:")
print(f"Baseline energy: {baseline_energy:.2e}")
print(f"Test energy: {test_energy:.2e}")
print(f"Difference energy: {diff_energy:.2e}")
print(f"Energy change: {(test_energy/baseline_energy - 1)*100:.1f}%")

## Create Spatial Grid and Calculate Propagation Distances

Generate a grid of points of interest within the structure and calculate guided wave propagation distances.

In [ ]:
# Create spatial grid for damage imaging
print("Building spatial grid...")

# Grid spacing (make it coarser for computational efficiency)
grid_spacing = 0.01  # 1 cm grid

# Build contained grid
grid_points, poi_mask, x_matrix, y_matrix = build_contained_grid_shm(
    border_comb, grid_spacing, grid_spacing
)

print(f"Grid created: {x_matrix.shape[0]} × {x_matrix.shape[1]} points")
print(f"Points inside structure: {len(grid_points)}")
print(f"Grid spacing: {grid_spacing*1000:.1f} mm")

# Calculate propagation distances for a subset of sensor pairs
n_pairs_analysis = min(5, pair_list.shape[0])  # Use subset for demonstration
analysis_pairs = pair_list[:n_pairs_analysis]

print(f"\nCalculating propagation distances for {n_pairs_analysis} sensor pairs...")
prop_distances = propagation_dist_2_points_shm(
    analysis_pairs, sensor_layout, grid_points
)

print(f"Propagation distance matrix shape: {prop_distances.shape}")
print(f"Distance range: {np.min(prop_distances):.3f} - {np.max(prop_distances):.3f} m")

# Convert distances to time-of-flight indices
time_indices = distance_2_index_shm(
    prop_distances, sample_rate, estimated_velocity, offset=0.0
)

print(f"\nTime-of-flight indices:")
print(f"Index range: {np.min(time_indices)} - {np.max(time_indices)}")
print(f"Maximum time-of-flight: {np.max(time_indices)/sample_rate*1000:.2f} ms")

# Visualize spatial grid and distances
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot 1: Spatial grid
ax1.scatter(grid_points[:, 0], grid_points[:, 1], 
           c='lightblue', s=1, alpha=0.5, label='Grid Points')
ax1.scatter(sensor_layout[:, 0], sensor_layout[:, 1], 
           c='red', s=100, marker='o', label='Sensors', zorder=3)

# Plot structure boundary
if border_comb:
    for border_segment in border_comb:
        if len(border_segment) > 1:
            ax1.plot(border_segment[:, 0], border_segment[:, 1], 'k-', linewidth=2)

ax1.set_xlabel('X Position (m)')
ax1.set_ylabel('Y Position (m)')
ax1.set_title('Spatial Grid for Damage Imaging')
ax1.legend()
ax1.set_aspect('equal')
ax1.grid(True, alpha=0.3)

# Plot 2: Distance distribution
ax2.hist(prop_distances.flatten(), bins=30, alpha=0.7, color='green')
ax2.set_xlabel('Propagation Distance (m)')
ax2.set_ylabel('Count')
ax2.set_title('Distribution of Propagation Distances')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Apply Matched Filtering

Demonstrate coherent and incoherent matched filtering on guided wave signals to detect scattered waves.

In [ ]:
# Apply matched filtering to detect guided waves
print("Applying matched filtering...")

# Calculate difference signal (test - baseline)
waveform_diff = waveform_test - waveform_base

# Use baseline signal as template for matched filtering
# Extract template from actuated sensor (typically first sensor)
template_length = min(100, n_time_points // 4)  # Use first part of signal
template = waveform_base[:template_length, 0]  # Use first sensor as template

print(f"Template length: {template_length} samples ({template_length/sample_rate*1000:.2f} ms)")

# Apply coherent matched filter to difference signal
coherent_result = coherent_matched_filter_shm(waveform_diff, template[:, np.newaxis])

# Apply incoherent matched filter to difference signal  
incoherent_result = incoherent_matched_filter_shm(waveform_diff, template[:, np.newaxis])

print(f"Coherent filter output shape: {coherent_result.shape}")
print(f"Incoherent filter output shape: {incoherent_result.shape}")

# Visualize matched filter results
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Template signal
template_time = np.arange(template_length) / sample_rate * 1000
axes[0, 0].plot(template_time, template, 'b-', linewidth=2)
axes[0, 0].set_xlabel('Time (ms)')
axes[0, 0].set_ylabel('Amplitude')
axes[0, 0].set_title('Template Signal (Baseline)')
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Difference signal (sensor 1)
axes[0, 1].plot(time_axis, waveform_diff[:, 0], 'r-', alpha=0.7)
axes[0, 1].set_xlabel('Time (ms)')
axes[0, 1].set_ylabel('Amplitude')
axes[0, 1].set_title('Difference Signal (Test - Baseline)')
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Coherent matched filter result
axes[1, 0].plot(time_axis, coherent_result[:, 0], 'g-', alpha=0.7)
axes[1, 0].set_xlabel('Time (ms)')
axes[1, 0].set_ylabel('Amplitude')
axes[1, 0].set_title('Coherent Matched Filter Output')
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Incoherent matched filter result
axes[1, 1].plot(time_axis, incoherent_result[:, 0], 'm-', alpha=0.7)
axes[1, 1].set_xlabel('Time (ms)')
axes[1, 1].set_ylabel('Magnitude')
axes[1, 1].set_title('Incoherent Matched Filter Output')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate signal energy metrics
baseline_energy = np.mean(np.sum(waveform_base**2, axis=0))
test_energy = np.mean(np.sum(waveform_test**2, axis=0))
diff_energy = np.mean(np.sum(waveform_diff**2, axis=0))

print(f"\nSignal Energy Analysis:")
print(f"Baseline energy: {baseline_energy:.2e}")
print(f"Test energy: {test_energy:.2e}")
print(f"Difference energy: {diff_energy:.2e}")
print(f"Energy change: {(test_energy/baseline_energy - 1)*100:.1f}%")

## Feature Extraction and Spatial Mapping

Extract damage-sensitive features at time-of-flight locations and map them to the spatial grid for damage visualization.

In [ ]:
# Extract features at time-of-flight indices
print("Extracting spatial damage features...")

# Initialize damage indicator array
damage_indicators = np.zeros((n_pairs_analysis, len(grid_points)))

# Extract features for each sensor pair and grid point
for pair_idx in range(n_pairs_analysis):
    # Get sensor indices
    sensor1_idx = analysis_pairs[pair_idx, 0] - 1  # Convert to 0-based
    sensor2_idx = analysis_pairs[pair_idx, 1] - 1
    
    # Use incoherent matched filter result (magnitude)
    signal1 = incoherent_result[:, sensor1_idx]
    signal2 = incoherent_result[:, sensor2_idx]
    
    for point_idx in range(len(grid_points)):
        # Get time-of-flight index for this pair-point combination
        tof_index = time_indices[pair_idx, point_idx]
        
        # Extract feature if index is valid
        if 0 <= tof_index < len(signal2):
            # Use maximum of both sensors
            feature_value = max(
                abs(signal1[tof_index]) if tof_index < len(signal1) else 0,
                abs(signal2[tof_index]) if tof_index < len(signal2) else 0
            )
            damage_indicators[pair_idx, point_idx] = feature_value

# Sum contributions from all sensor pairs
total_damage_indicator = np.sum(damage_indicators, axis=0)

print(f"Feature extraction completed")
print(f"Damage indicator range: {np.min(total_damage_indicator):.2e} - {np.max(total_damage_indicator):.2e}")
print(f"Non-zero indicators: {np.sum(total_damage_indicator > 0)} / {len(total_damage_indicator)}")

# Map 1D results back to 2D spatial grid
damage_map_2d = fill_2d_map_shm(total_damage_indicator, poi_mask)

print(f"2D damage map shape: {damage_map_2d.shape}")

# Create comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Individual sensor pair contributions
im1 = axes[0, 0].imshow(damage_map_2d, extent=[x_matrix.min(), x_matrix.max(), 
                                               y_matrix.min(), y_matrix.max()],
                        origin='lower', cmap='hot', alpha=0.8)
axes[0, 0].scatter(sensor_layout[:, 0], sensor_layout[:, 1], 
                  c='blue', s=100, marker='o', label='Sensors')
axes[0, 0].set_xlabel('X Position (m)')
axes[0, 0].set_ylabel('Y Position (m)')
axes[0, 0].set_title('Damage Indicator Map')
axes[0, 0].legend()
plt.colorbar(im1, ax=axes[0, 0], label='Damage Indicator')

# Plot 2: Sensor pair contributions histogram
pair_contributions = np.sum(damage_indicators, axis=1)
axes[0, 1].bar(range(n_pairs_analysis), pair_contributions, alpha=0.7)
axes[0, 1].set_xlabel('Sensor Pair Index')
axes[0, 1].set_ylabel('Total Contribution')
axes[0, 1].set_title('Contributions by Sensor Pair')
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Cross-section through damage map
center_row = damage_map_2d.shape[0] // 2
x_cross_section = x_matrix[center_row, :]
damage_cross_section = damage_map_2d[center_row, :]

axes[1, 0].plot(x_cross_section, damage_cross_section, 'r-', linewidth=2)
axes[1, 0].set_xlabel('X Position (m)')
axes[1, 0].set_ylabel('Damage Indicator')
axes[1, 0].set_title('Cross-Section Through Damage Map')
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Statistics
stats_data = [
    baseline_energy,
    test_energy, 
    diff_energy,
    np.max(total_damage_indicator)
]
stats_labels = ['Baseline\nEnergy', 'Test\nEnergy', 'Diff\nEnergy', 'Max Damage\nIndicator']

# Normalize for display
stats_normalized = [s / np.max(stats_data) for s in stats_data]

bars = axes[1, 1].bar(stats_labels, stats_normalized, 
                     color=['blue', 'green', 'red', 'orange'], alpha=0.7)
axes[1, 1].set_ylabel('Normalized Value')
axes[1, 1].set_title('Signal Analysis Summary')
axes[1, 1].set_ylim(0, 1.1)

# Add value labels on bars
for bar, value in zip(bars, stats_data):
    height = bar.get_height()
    axes[1, 1].text(bar.get_x() + bar.get_width()/2., height + 0.02,
                    f'{value:.1e}', ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.show()

# Summary statistics
print(f"\nActive Sensing Analysis Summary:")
print(f"Estimated group velocity: {estimated_velocity:.0f} m/s")
print(f"Maximum damage indicator: {np.max(total_damage_indicator):.2e}")
print(f"Average damage indicator: {np.mean(total_damage_indicator):.2e}")
print(f"Spatial resolution: {grid_spacing*1000:.1f} mm")
print(f"Analysis covered {n_pairs_analysis} sensor pairs")

## Summary

This example demonstrated active sensing feature extraction using guided wave analysis for structural health monitoring. The approach successfully implemented:

**Key Results:**

- **Guided Wave Analysis**: Processed ultrasonic measurements from sensor networks
- **Velocity Estimation**: Determined wave propagation characteristics
- **Matched Filtering**: Applied coherent and incoherent detection methods
- **Spatial Mapping**: Created 2D damage indicator maps

**Technical Achievements:**
1. **Signal Processing**: Implemented matched filtering for wave detection
2. **Geometric Analysis**: Calculated propagation paths and time-of-flight
3. **Feature Extraction**: Extracted damage indicators at specific locations
4. **Visualization**: Created comprehensive spatial damage maps

**Advantages of Active Sensing:**
1. **High Sensitivity**: Can detect small defects through wave scattering
2. **Spatial Localization**: Provides damage location information
3. **Controlled Excitation**: Uses known input signals for optimal detection
4. **Wide Coverage**: Single sensor network can monitor large areas

**Considerations:**
1. **Computational Complexity**: Requires significant processing for real-time applications
2. **Environmental Sensitivity**: Temperature and loading can affect wave propagation
3. **Geometric Constraints**: Line-of-sight limitations affect coverage
4. **Hardware Requirements**: Needs specialized actuators and high-speed acquisition

**For Structural Health Monitoring:**

Active sensing provides a powerful approach for damage detection in complex structures. The combination of guided wave propagation physics with advanced signal processing enables both detection and localization of structural damage. This implementation provides a foundation for developing practical active sensing monitoring systems.

**See also:**
- [Outlier Detection based on Principal Component Analysis](../basic/pca_outlier_detection.ipynb)
- [Nonparametric Outlier Detection](nonparametric_outlier_detection.ipynb)
- [Semi-Parametric Outlier Detection](semiparametric_outlier_detection.ipynb)